# Прогнозирование временных рядов

Прогнозирование временных рядов — это важная задача в аналитике данных, которая помогает предсказывать будущее поведение данных на основе исторических данных. В этом блоке мы рассмотрим основы анализа временных рядов, изучим, как работать с ними в Python, и познакомимся с основными методами оценки точности прогнозов.

---

## Введение в анализ временных рядов

### Что такое временные ряды и их использование

**Временные ряды** — это последовательность наблюдений, записанных через равные промежутки времени. Они широко используются для анализа данных, которые зависят от времени, таких как прогнозирование продаж, температуры, финансовых показателей, спроса на продукцию и т.д.

Примеры применения временных рядов:
- **Финансовый анализ**: прогнозирование цен на акции или обменных курсов.
- **Производственные процессы**: прогнозирование объёмов выпуска продукции.
- **Экономика**: анализ и прогнозирование макроэкономических показателей.
- **Прогнозирование спроса**: планирование запасов на основе данных о продажах.

### Основные методы прогнозирования

1. **Простое сглаживание (Simple Moving Average)**:
   - Метод заключается в усреднении ряда данных за определённый промежуток времени.
   - Пример: среднее значение продаж за последние 3 месяца.

2. **Экспоненциальное сглаживание (Exponential Smoothing)**:
   - Усреднение данных с учётом весов, придаваемых более свежим данным.
   - Пример: модель **Holt-Winters**, которая учитывает как тренд, так и сезонность данных.

3. **ARIMA (AutoRegressive Integrated Moving Average)**:
   - Популярная модель для прогнозирования временных рядов, которая включает авторегрессию (AR), интегрированную компоненту (I) и скользящее среднее (MA).
   - Применяется для прогнозирования данных с трендами и сезонными колебаниями.

4. **Prophet**:
   - Модель, разработанная Facebook, специально для работы с временными рядами. Хорошо справляется с данными, содержащими тренды, сезонность и аномалии.

5. **TSFresh и Pmdarima**:
   - **TSFresh** используется для извлечения признаков из временных рядов для дальнейшего анализа и машинного обучения.
   - **Pmdarima** автоматически подбирает параметры ARIMA и выполняет прогнозирование.

---

## Работа с временными рядами в Python

#### Введение в библиотеки Pandas для работы с временными рядами

**Pandas** предоставляет множество удобных инструментов для работы с временными рядами. Среди них:
- Индексация по дате и времени.
- Вычисление скользящего среднего.
- Ресемплирование данных (например, агрегирование по дням, неделям, месяцам).

##### Пример работы с временными рядами в Pandas:

```python
import pandas as pd
import numpy as np

# Загрузка данных
df = pd.read_excel('timeseries.xlsx', index_col='date', parse_dates=['date'])

# Вывод первых строк с индексом времени
df.head()
```

Вывод графика

```python
# Построение графика временного ряда
df['value'].plot(title='Объем, тыс. тонн', figsize=(17,4))
```

### Пояснение:
- **`parse_dates=['Date']`** — преобразует столбец с датами в формат `datetime`.
- **`index_col='Date'`** — делает столбец с датами индексом DataFrame.
- **`plot()`** — построение графика временного ряда.

## Поиск выбросов во временных рядах с учётом сезонных колебаний

Временные ряды могут иметь как случайные выбросы, так и выбросы, связанные с сезонными колебаниями. Выбросы, не принимающие во внимание сезонность, могут ошибочно считаться аномалиями. В этом блоке рассмотрим, как искать выбросы во временных рядах с учётом сезонных колебаний и как корректировать их.

### Шаг 1: Поиск выбросов с учётом сезонности

Для корректного поиска выбросов во временных рядах с выраженной сезонностью часто применяются специализированные методы. Вот несколько подходов:

1. **Декомпозиция временного ряда** (с использованием библиотек **statsmodels** или **Prophet**) позволяет отделить сезонную составляющую от тренда и случайных колебаний.
2. **Использование скользящего среднего с учётом сезонных изменений**.
3. **Поиск выбросов с помощью локальной регрессии (LOESS)**.

#### 1.1 Декомпозиция временного ряда

Декомпозиция позволяет разделить временной ряд на три компонента:
- **Тренд** — долгосрочные изменения во времени.
- **Сезонность** — повторяющиеся циклические колебания.
- **Остатки** — случайные флуктуации и выбросы.

Сначала разделим временной ряд с помощью декомпозиции, а затем поищем выбросы в остатках.

```python

import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Декомпозиция временного ряда (добавьте свой временной ряд в 'Cost_USD')
decomposition = seasonal_decompose(df['value'], model='additive', period=12)

# Визуализация компонентов декомпозиции
decomposition.plot()
plt.show()

# Выделение компоненты остатков (residuals)
residuals = decomposition.resid

# Определение выбросов в остатках с помощью метода межквартильного размаха (IQR)
Q1 = residuals.quantile(0.25)
Q3 = residuals.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Поиск выбросов в остатках
outliers = residuals[(residuals < lower_bound) | (residuals > upper_bound)]

# Вывод выбросов
print('Выбросы:')
outliers
```

### Пояснение:
- **`seasonal_decompose()`** — функция для декомпозиции временного ряда на тренд, сезонность и остатки. 
- **Остатки** содержат случайные флуктуации и выбросы, поэтому именно они используются для поиска аномалий.

#### 1.2 Скользящее среднее с учётом сезонных изменений

Другим подходом к поиску выбросов является использование скользящего среднего с учётом сезонных колебаний. Важно выбрать подходящий размер окна, который соответствует длине сезонного цикла.

```python
# Скользящее среднее с окном, равным длине сезонного цикла (например, 12 месяцев)
rolling_mean = df['value'].rolling(window=12).mean()
rolling_std = df['value'].rolling(window=12).std()

# Поиск выбросов на основе скользящего среднего и стандартного отклонения
outliers = df[(df['value'] > rolling_mean + 2 * rolling_std) | 
              (df['value'] < rolling_mean - 2 * rolling_std)]

# Вывод выбросов
print(outliers[['value']])
```

### Пояснение:
- **`rolling(window=12)`** — скользящее среднее и стандартное отклонение с окном, равным 12 наблюдениям (например, 12 месяцев для годового цикла).

#### 1.3 Поиск выбросов с использованием локальной регрессии (LOESS)

Локальная регрессия (LOESS) — это метод, который позволяет сгладить данные с учётом их локальной структуры, включая сезонность. Этот метод помогает найти выбросы, которые отклоняются от локальных трендов.

```python
from statsmodels.nonparametric.smoothers_lowess import lowess

# Применение метода LOESS для сглаживания временного ряда
smoothed = lowess(df['value'], df.index, frac=0.1)

# Вычисление остатков (разница между исходными данными и сглаженными)
residuals = df['value'] - smoothed[:, 1]

# Поиск выбросов в остатках с использованием метода IQR
Q1 = pd.Series(residuals).quantile(0.25)
Q3 = pd.Series(residuals).quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = residuals[(residuals < lower_bound) | (residuals > upper_bound)]

# Вывод выбросов
outliers
```

### Пояснение:
- **`lowess()`** — сглаживает временной ряд с помощью локальной регрессии.
- Выбросы затем находятся в остатках, используя метод IQR.

### Шаг 2: Рекомендации по исправлению выбросов

#### 2.1 Коррекция выбросов с учётом сезонных колебаний

##### 2.1.1 Удаление выбросов в остатках

Если выбросы являются результатом случайных аномалий и не имеют важного значения для тренда или сезонности, их можно удалить.

```python
# Удаление выбросов в остатках
df_cleaned = df[(residuals >= lower_bound) & (residuals <= upper_bound)]
```

#### Рекомендации:
- Удаляйте выбросы в остатках после учёта сезонности и тренда, чтобы не потерять важные сезонные или трендовые колебания.

#### 2.1.2 Замена выбросов на сезонные значения

Если выбросы являются аномальными отклонениями, но сами данные важны для анализа, можно заменить их на соответствующие сезонные значения (или на среднее для каждого сезона).

```python
# Предположим, что 'decomposition.seasonal' - это ряд сезонных значений, имеющий ту же длину, что и исходный DataFrame.
# Если это временной ряд, используем соответствующее значение из сезонной компоненты.

# Замена выбросов на соответствующее сезонное значение
df['value_corrected'] = df.apply(
    lambda row: decomposition.seasonal[row.name] if (row['value'] < lower_bound or row['value'] > upper_bound) else row['value'],
    axis=1
)

df['value_corrected']
```

#### Рекомендации:
- Замена выбросов сезонными значениями сохраняет целостность данных и учитывает сезонные колебания.

#### 2.2 Использование логарифмического преобразования

Логарифмическое преобразование может уменьшить влияние выбросов, особенно когда выбросы значительно отклоняются по величине.

```python
# Применение логарифмического преобразования к данным
df['Log_value'] = np.log(df['value'])
df['Log_value'].plot()
```

#### Рекомендации:
- Логарифмическое преобразование часто помогает, если выбросы пропорциональны значению переменной и связаны с изменением масштаба.

### Заключение

Для поиска выбросов в сезонных временных рядах лучше всего использовать методы, которые учитывают структуру данных, такие как декомпозиция или скользящее среднее с сезонным окном. Исправление выбросов может включать удаление аномалий, замену сезонными средними значениями или использование логарифмического преобразования для сглаживания отклонений. 

1. **Декомпозиция** и **LOESS** — позволяют точно выделить остатки для поиска аномалий.
2. **Скользящее среднее с учётом сезонности** — позволяет находить выбросы в сезонных рядах.
3. **Исправление**: удаление, замена на сезонные значения или логарифмическое преобразование.

Выбор метода зависит от задачи и структуры данных.

## Использование библиотеки Prophet для прогнозирования

Prophet — это мощная библиотека для прогнозирования временных рядов, которая автоматически учитывает тренды и сезонность данных.

##### Установка Prophet:

```bash
!pip -q install prophet
```

##### Пример использования Prophet:

```python
import pandas as pd
import numpy as np

# Загрузка данных
#
# !! Не читаем, если ряд уже отредактировали, тогда используем переменную с откоректированным рядом
#
df = pd.read_excel('timeseries.xlsx', index_col='date', parse_dates=['date'])

# Вывод первых строк с индексом времени
df.head()


from prophet import Prophet

# Подготовка данных для Prophet
df_prophet = df[['date', 'value']].reset_index().rename(columns={'date': 'ds', 'value': 'y'})
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

# Создание модели Prophet
model = Prophet()
model.fit(df_prophet)

# Прогнозирование на следующие 30 дней
future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)

# Визуализация прогноза
model.plot(forecast)
plt.show()
```

### Пояснение:
- **`ds`** — стандартное имя для столбца с датами в Prophet.
- **`y`** — значение временного ряда, которое нужно прогнозировать.
- **`make_future_dataframe(periods=30)`** — создаёт прогноз на следующие 30 дней.

Посмотреть прогноз можно вызвав переменную `forecast`

Или сохранить в Excel файл

```python
forecast.to_excel('forecast_2025.xlsx')
```

## Модель Хольта-Винтерса

Модель Хольта-Винтерса (Holt-Winters) — это метод временного ряда, который используется для прогнозирования данных с трендом и сезонностью. Давайте рассмотрим пример, как использовать эту модель для прогнозирования.

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Обучение модели Хольта-Винтерса
df=df.resample('M').mean()
model = ExponentialSmoothing(df['value'], seasonal='mul', seasonal_periods=12, use_boxcox=False)
hw_model = model.fit(optimized=True, remove_bias=False)

# Прогнозирование на следующие 12 месяцев
forecast_hw = hw_model.forecast(12)

# Визуализация результатов
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['value'], label='Исторические данные')
plt.plot(pd.date_range(start=df.index[-1] + pd.DateOffset(1), periods=12, freq='M'), forecast_hw, label='Прогноз', color='orange')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.title('Прогноз с использованием модели Хольта-Винтерса')
plt.legend()
plt.show()
```

**Объяснение кода:**

Импорт библиотек: Используем pandas для работы с временными рядами, numpy для генерации данных, matplotlib для визуализации и ExponentialSmoothing из statsmodels для применения модели Хольта-Винтерса.

Обучение модели Хольта-Винтерса:
- Параметр seasonal='add' указывает, что сезонность в данных аддитивная. Если ваша сезонность мультипликативная, используйте seasonal='mul'.
- seasonal_periods=12 указывает, что сезонность повторяется каждые 12 месяцев.

Прогнозирование: Используем метод forecast для предсказания на 12 будущих периодов.

Визуализация: Отображаем как исторические данные, так и прогноз, чтобы увидеть, как модель предсказывает будущие значения.

Этот пример демонстрирует, как использовать модель Хольта-Винтерса для прогнозирования временных рядов с сезонностью.

## Оценка моделей временных рядов

После построения модели важно оценить её точность, используя различные метрики. В прогнозировании временных рядов наиболее часто применяются следующие метрики:

1. **MAE (Mean Absolute Error)** — средняя абсолютная ошибка. Измеряет среднее абсолютное отклонение прогнозов от фактических значений.

   Формула:
   $$
   MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
   $$

2. **RMSE (Root Mean Squared Error)** — среднеквадратичная ошибка. Оценка средней ошибки прогнозирования с увеличенным влиянием больших отклонений.

   Формула:
   $$
   RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
   $$

3. **MAPE (Mean Absolute Percentage Error)** — средняя абсолютная процентная ошибка. Показывает, на сколько процентов в среднем прогноз отличается от фактического значения.

   Формула:
   $$
   MAPE = \frac{1}{n} \sum_{i=1}^{n} \left( \frac{|y_i - \hat{y}_i|}{y_i} \right) \times 100
   $$

#### Пример расчёта метрик:

```python
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

# Истинные значения и прогнозы (например, для Prophet)
y_true = df['value'][-12:].values  # последние 30 значений для оценки
y_pred = forecast['yhat'][-12:].values  # прогнозируемые значения Prophet

# MAE
mae = mean_absolute_error(y_true, y_pred)

# RMSE
rmse = root_mean_squared_error(y_true, y_pred)

# MAPE
mape = (abs((y_true - y_pred) / y_true)).mean() * 100

# Вывод метрик
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')
```

### Пояснение:
- **`mean_absolute_error()`** — вычисляет среднюю абсолютную ошибку.
- **`mean_squared_error(squared=False)`** — вычисляет RMSE.
- **`MAPE`** — рассчитывается вручную, поскольку она не встроена в `sklearn`.

### Для Хольта-Винтерса

Чтобы рассчитать метрики ошибки, такие как **MAE** (Mean Absolute Error), **MAPE** (Mean Absolute Percentage Error), и **RMSE** (Root Mean Squared Error) без разделения данных на обучающие и тестовые, можно использовать следующую стратегию:

1. **Сравнить прогнозируемые значения с последними известными данными в исходной серии**. Для этого вы можете взять последние 12 значений из временного ряда и использовать их в качестве "тестовых" данных для оценки точности модели.

### Пример кода для расчета метрик ошибки:

```python
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Предположим, что у вас есть 12 последних известных значений для сравнения
n_test = 12  # Количество значений для тестирования
y_true = df['value'][-n_test:]  # Последние 12 известных значений в исходном временном ряде
y_pred = hw_model.fittedvalues[-n_test:]  # Соответствующие предсказания модели

# Рассчитаем метрики ошибки
mae = mean_absolute_error(y_true, y_pred)
mse = root_mean_squared_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Вывод результатов
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {mse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
```

### Объяснение:
1. **y_true**: Это последние 12 известных значений из вашего временного ряда, которые мы используем для сравнения.
2. **y_pred**: Это предсказанные значения, полученные моделью Хольта-Винтерса. Мы используем `hw_model.fittedvalues` для извлечения предсказаний, которые соответствуют историческим данным.
3. **MAE, RMSE, MAPE**: Вычисляются с использованием библиотек `sklearn` и `numpy`.

### Примечание:
- `hw_model.fittedvalues` предоставляет предсказания модели на основе обучающих данных. Мы сравниваем последние известные значения с соответствующими предсказаниями.
- Этот подход позволяет оценить точность модели, даже если вы не разделяли данные явно на тренировочные и тестовые.

Теперь вы сможете получить оценку ошибок модели и понять, насколько хорошо она справляется с прогнозированием.

## Использование библиотеки Pmdarima для ARIMA

Pmdarima — это удобная библиотека для работы с моделями временных рядов, которая автоматизирует процесс построения моделей ARIMA. Она упрощает выбор оптимальных параметров модели с помощью функции auto_arima, что значительно облегчает анализ временных рядов.

**Pmdarima** автоматизирует процесс выбора параметров для модели ARIMA.

##### Установка Pmdarima:

```bash
!pip install -q pmdarima
```

##### Пример использования Pmdarima:

```python
import pandas as pd
import matplotlib.pyplot as plt
from pmdarima import auto_arima

model_arima = auto_arima(
    df['value'],
    start_p=1, start_q=1,
    max_p=5, max_q=5,
    seasonal=True, m=12,
    d=True,  # Автоматическое определение дифференцирования
    stepwise=False,  # Перебор всех вариантов
    trace=True,  # Вывод информации о процессе перебора
    error_action='ignore',
    suppress_warnings=True
)

# Прогнозирование на следующие 12 периодов
forecast_arima = model_arima.predict(n_periods=12)

# Создание индексов для прогнозов
forecast_index = pd.date_range(start=df.index[-1] + pd.DateOffset(1), periods=12, freq='M')

# Визуализация результатов
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['value'], label='Исторические данные')
plt.plot(forecast_index, forecast_arima, label='Прогноз', color='orange')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.title('Прогноз с использованием модели ARIMA')
plt.legend()
plt.show()

```

### Пояснение:
- **`auto_arima()`** — автоматически подбирает параметры модели ARIMA.
- **`seasonal=True`** и **`m=12`** — задают сезонность данных с периодом 12 (например, месячные данные).

Модель Arima имеет свои встроенные функции контроля качества модели.

```python
model_arima.plot_diagnostics()
```

Давайте разберем результаты диагностики модели ARIMA, представленные на изображении. Эти графики помогут вам понять, насколько хорошо модель справляется с анализом временного ряда и где могут возникать проблемы.

##### 1. **Standardized Residual (Стандартизированные остатки)**
   - Этот график показывает остатки модели (разницу между предсказанными значениями и фактическими данными), стандартизированные до нормального распределения.
   - **Как интерпретировать:** Остатки должны вести себя случайно, без заметных паттернов. Если вы видите четкие повторяющиеся структуры или тренды, это может указывать на недооценку сезонности или трендов в данных.

##### 2. **Histogram plus Estimated Density (Гистограмма остатков с оценкой плотности)**
   - Гистограмма показывает распределение остатков, наложенное на оценочную плотность распределения (оранжевая линия) и нормальное распределение (зеленая линия).
   - **Как интерпретировать:** Если гистограмма и оценочная плотность хорошо совпадают с кривой нормального распределения, это хороший знак. Однако, если распределение остатков сильно отличается от нормального, это может указывать на ошибки в модели или выбросы в данных.

##### 3. **QQ-Plot (График квантиль-квантиль)**
   - Этот график показывает, насколько остатки соответствуют нормальному распределению. Точки должны ложиться вдоль диагональной линии, если остатки нормально распределены.
   - **Как интерпретировать:** Если точки отклоняются от диагонали, это указывает на отклонения от нормальности. Сильные отклонения в начале или конце линии могут указывать на наличие выбросов или неучтенных структур в данных.

##### 4. **Correlogram (Коррелограмма остатков)**
   - Этот график показывает автокорреляцию остатков на разных лагах (временных промежутках). Автокорреляция измеряет, насколько текущие остатки связаны с прошлыми значениями.
   - **Как интерпретировать:** Остатки должны быть некоррелированными, то есть точки должны находиться внутри доверительных интервалов (голубые линии). Если есть значительная автокорреляция на определенных лагах, это может означать, что модель не учла некоторые зависимости во временном ряде.

---

##### **Инструкция по трактовке результатов:**
1. **Проверьте график стандартизированных остатков:** Если видны явные паттерны, попробуйте пересмотреть параметры модели ARIMA, чтобы лучше захватить тренды или сезонность.
2. **Оцените гистограмму:** Распределение остатков должно быть близко к нормальному. Если это не так, возможно, стоит трансформировать данные (например, логарифмировать) или попробовать другую модель.
3. **Посмотрите на QQ-график:** Если точки сильно отклоняются от диагональной линии, подумайте о трансформациях данных или пересмотре предположений модели.
4. **Анализируйте коррелограмму:** Если остатки автокоррелированы (точки выходят за доверительные интервалы), модель может быть недостаточно сложной. Рассмотрите добавление дополнительных лагов или изменение параметров модели.

Эти шаги помогут вам понять, насколько модель хорошо подходит для данных и где нужно внести изменения для улучшения прогнозов.

### Шаги для вычисления MSE и MAPE для ряда
1. **Разделим данные на обучающую часть и оставим конец ряда для проверки (называем их тестовыми данными).**
2. **Сделаем прогноз и сравним его с реальными данными, чтобы рассчитать RMSE и MAPE.**

### Пример кода:
```python
import numpy as np
from sklearn.metrics import mean_squared_error

# Длина временного ряда
n_total = len(df)
n_test = 12  # Количество точек для тестирования (можете выбрать другое значение)

# Разделение на обучающую и тестовую выборки
train = df['value'][:n_total - n_test]  # Все данные, кроме последних n_test точек
test = df['value'][-n_test:]  # Последние n_test точек

# Обучение модели ARIMA на обучающих данных
from pmdarima import auto_arima
model_arima = auto_arima(train, seasonal=True, m=12)

# Прогнозирование на тестовых данных
forecast_arima = model_arima.predict(n_periods=n_test)

# Вычисление MSE (среднеквадратичная ошибка)
mse = root_mean_squared_error(test, forecast_arima)
print(f"Root Mean Squared Error (RMSE): {mse:.2f}")

# Вычисление MAPE (средняя абсолютная процентная ошибка)
mape = np.mean(np.abs((test - forecast_arima) / test)) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
```

### Объяснение:
1. **Разделение данных:** Мы используем всю серию данных `df['value']` и делим её на обучающую часть `train` и тестовую часть `test`, чтобы иметь возможность оценить точность модели.
2. **Обучение модели:** Мы используем данные `train` для подбора модели ARIMA с помощью `auto_arima`.
3. **Прогнозирование:** Прогнозируем на количество точек, равное длине тестового набора данных (`n_test`).
4. **Оценка точности:** Вычисляем RMSE и MAPE, сравнивая предсказанные значения `forecast_arima` с реальными значениями `test`.

Теперь вы можете увидеть, насколько хорошо модель предсказывает данные и получить оценку её точности.

**Предложение**: выполните преобразование данных для сглаживания пиков 

```python
# Пример логарифмической трансформации
df['value_log'] = np.log(df['value'])```

Постройте модель на новых данных, для этого изменить название целевого столбца.

Для возврата к нормальной размерности:

```python
# Предположим, что forecast_log - это ваш прогноз в логарифмической размерности
forecast_normal = np.exp(forecast_log)  # Возвращаем данные к исходной размерности
```

## Заключение

1. **Основы временных рядов**: Временные ряды используются для анализа данных, зависящих от времени, с использованием различных методов прогнозирования (от простого сглаживания до сложных моделей).
2. **Работа с временными рядами в Python**: Мы рассмотрели библиотеки Pandas для работы с временными рядами и такие инструменты для прогнозирования, как Prophet и Pmdarima.
3. **Оценка точности моделей**: MAE, RMSE и MAPE — основные метрики, используемые для оценки качества прогноза.

Эти знания помогут вам эффективно анализировать и прогнозировать временные ряды.

# Задание

Выберите в датасете иной ряд для прогнозирования (другую группу грузов).

Проанализируйте, очистите ряд и постройте прогноз двумя методами. 

Сравните результаты и выберите наиболее точный. 